# VGG16

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 5GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
model_id = "vgg16_test"
train_dir = "../data/sample-ub-9977"
batch_size = 128
image_size = 128

In [4]:
# Data Augmentation Parameters
image_generator = ImageDataGenerator(
    validation_split = 0.2,
    rescale = 1./255,
    rotation_range = 45,
    horizontal_flip = True,
    zoom_range = 0.4,
    fill_mode = 'nearest'
)

In [5]:
train_data_gen = image_generator.flow_from_directory(
    directory = train_dir,
    batch_size = batch_size,
    target_size = (image_size, image_size),
    class_mode = "categorical",
    subset = "training"
)

Found 8001 images belonging to 42 classes.


In [6]:
validation_data_gen = image_generator.flow_from_directory(
    directory = train_dir,
    batch_size = batch_size,
    target_size = (image_size, image_size),
    class_mode = "categorical",
    subset = "validation"
)

Found 1976 images belonging to 42 classes.


In [11]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, kernel_size = (5, 5), input_shape = (image_size, image_size, 3), activation = "relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D((2, 2)))

model.add(keras.layers.Conv2D(64, kernel_size = (5, 5), activation = "relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D((2, 2)))

model.add(keras.layers.Conv2D(128, kernel_size = (3, 3), activation = "relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D((2, 2)))

model.add(keras.layers.Conv2D(256, kernel_size = (3, 3), activation = "relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D((2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation = "relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(train_data_gen.num_classes, activation = "softmax"))

In [12]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.CategoricalCrossentropy(),
    metrics = ["accuracy"]
)

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 124, 124, 32)      2432      
_________________________________________________________________
batch_normalization_4 (Batch (None, 124, 124, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 58, 58, 64)        51264     
_________________________________________________________________
batch_normalization_5 (Batch (None, 58, 58, 64)        256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 128)      

In [14]:
history = model.fit(
    train_data_gen,
    epochs = 20,
    steps_per_epoch = train_data_gen.n // batch_size,
    validation_data = validation_data_gen,
    validation_steps = validation_data_gen.n // batch_size,
    callbacks = [
        keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True, verbose = 1)
#         keras.callbacks.CSVLogger(MODEL_DIR + "{}.csv".format(model_id))
    ]
)

Epoch 1/20
62/62 [==============================] - 92s 1s/step - loss: 4.2408 - accuracy: 0.0526 - val_loss: 3.7642 - val_accuracy: 0.0250
Epoch 2/20
62/62 [==============================] - 93s 1s/step - loss: 3.6559 - accuracy: 0.0600 - val_loss: 3.7687 - val_accuracy: 0.0365
Epoch 3/20
62/62 [==============================] - 93s 1s/step - loss: 3.5849 - accuracy: 0.0784 - val_loss: 3.9250 - val_accuracy: 0.0359
Epoch 4/20
62/62 [==============================] - 93s 2s/step - loss: 3.5308 - accuracy: 0.0895 - val_loss: 3.7630 - val_accuracy: 0.0562
Epoch 5/20
52/62 [========================>.....] - ETA: 12s - loss: 3.4429 - accuracy: 0.1047

InternalError: GPU sync failed

In [ ]:
acc = history.history["accuracy"]
loss = history.history["loss"]
validation_acc = history.history["val_accuracy"]
validation_loss = history.history["val_loss"]

epochs_range = range(len(history.history["accuracy"]))

plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = "Training Accuracy")
plt.plot(epochs_range, validation_acc, label = "Validation Accuracy")
plt.legend(loc = "lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = "Training Loss")
plt.plot(epochs_range, validation_loss, label = "Validation Loss")
plt.legend(loc = "lower right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
# model.save(MODEL_DIR + "{}.h5".format(model_id), overwrite = False)